# 第21章：Evaluation & Testing（评估与测试）

## 学习目标

本章将学习：
1. 为什么需要评估LLM应用
2. 评估 vs 测试的区别
3. 创建测试数据集
4. RAG评估指标（相关性、准确性、忠实度）
5. Agent评估方法（轨迹评估）
6. LLM-as-Judge评估器
7. 自定义评估器
8. 使用pytest进行测试
9. 评估最佳实践

---

## 为什么需要评估？

### LLM应用的挑战

LLM输出是**非确定性**的，这使得评估质量变得困难：

| 传统软件 | LLM应用 |
|---------|---------|
| 输入相同，输出确定 | 输入相同，输出可能不同 |
| 可以用单元测试验证 | 需要评估指标衡量 |
| 正确性是二元的（对/错） | 质量是连续的（好/中/差） |
| 容易自动化测试 | 需要更复杂的评估方法 |

### 评估的价值

✅ **确保质量** - 持续监控系统性能  
✅ **对比方案** - 比较不同的模型、提示词、架构  
✅ **发现问题** - 及早发现退化和错误  
✅ **指导优化** - 数据驱动的改进决策  
✅ **建立信心** - 为生产部署提供依据

---

## 核心概念

### 1. 评估 vs 测试

| 维度 | 评估（Evaluation） | 测试（Testing） |
|------|-------------------|----------------|
| **目的** | 衡量性能 | 断言正确性 |
| **结果** | 指标分数（连续值） | 通过/失败（二元） |
| **标准** | 相对的、可以模糊 | 绝对的、必须明确 |
| **用途** | 对比不同版本 | 确保基本功能 |
| **部署** | 分数低可以部署 | 测试失败不能部署 |

**关键**：评估指标可以转换为测试（如回归测试：新版本必须优于基线）

### 2. 评估工作流

```
1. 创建数据集
   ↓
2. 运行应用
   ↓
3. 使用评估器打分
   ↓
4. 分析结果
   ↓
5. 迭代改进
```

### 3. 评估器类型

| 类型 | 说明 | 示例 |
|------|------|------|
| **基于规则** | 简单函数检查 | 长度检查、格式验证 |
| **基于参考** | 与标准答案对比 | 准确性、一致性 |
| **LLM-as-Judge** | 用LLM评估输出 | 相关性、有用性 |
| **人工评估** | 人类专家打分 | 主观质量、创意性 |


In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")
print("✓ 准备学习评估与测试")

# 注意：本章重点讲解评估概念和方法
# LangSmith的在线评估功能将在第22章详细介绍

环境配置完成！
模型: gpt-4.1-mini
✓ 准备学习评估与测试


## 1. 创建测试数据集

### 核心价值

测试数据集是评估的基础，包含：
- **输入（Inputs）**：问题或查询
- **预期输出（Expected Outputs）**：参考答案（可选）
- **元数据（Metadata）**：额外信息

### 数据集类型

| 类型 | 说明 | 适用场景 |
|------|------|---------|
| **Golden Dataset** | 人工标注的高质量数据 | 核心功能测试 |
| **Synthetic Dataset** | 自动生成的数据 | 大规模测试 |
| **Production Data** | 从生产环境采样 | 真实场景测试 |

### 数据集示例

```python
# 简单的测试数据集
test_dataset = [
    {
        "input": "什么是Python？",
        "expected_output": "Python是一种高级编程语言...",
        "metadata": {"category": "basic", "difficulty": "easy"}
    },
    {
        "input": "如何安装LangChain？",
        "expected_output": "使用pip install langchain...",
        "metadata": {"category": "setup", "difficulty": "easy"}
    }
]
```

In [2]:
print("【演示1：创建测试数据集】\n")

# 创建一个简单的QA测试数据集
qa_dataset = [
    {
        "question": "Python是什么类型的语言？",
        "expected_answer": "Python是一种高级、解释型、面向对象的编程语言",
        "category": "basic"
    },
    {
        "question": "LangChain的主要用途是什么？",
        "expected_answer": "LangChain用于构建基于大语言模型的应用程序",
        "category": "framework"
    },
    {
        "question": "什么是RAG？",
        "expected_answer": "RAG（Retrieval Augmented Generation）是检索增强生成技术",
        "category": "concept"
    }
]

print(f"✓ 创建了包含 {len(qa_dataset)} 个样本的测试数据集\n")

# 查看数据集结构
for i, example in enumerate(qa_dataset, 1):
    print(f"样本 {i}:")
    print(f"  问题: {example['question']}")
    print(f"  预期答案: {example['expected_answer'][:30]}...")
    print(f"  类别: {example['category']}\n")

print("💡 数据集设计要点：")
print("  1. 覆盖核心功能和边界情况")
print("  2. 包含不同难度和类型的样本")
print("  3. 预期答案不需要完全匹配，但要包含关键信息")
print("  4. 添加元数据便于分类分析")

【演示1：创建测试数据集】

✓ 创建了包含 3 个样本的测试数据集

样本 1:
  问题: Python是什么类型的语言？
  预期答案: Python是一种高级、解释型、面向对象的编程语言...
  类别: basic

样本 2:
  问题: LangChain的主要用途是什么？
  预期答案: LangChain用于构建基于大语言模型的应用程序...
  类别: framework

样本 3:
  问题: 什么是RAG？
  预期答案: RAG（Retrieval Augmented Genera...
  类别: concept

💡 数据集设计要点：
  1. 覆盖核心功能和边界情况
  2. 包含不同难度和类型的样本
  3. 预期答案不需要完全匹配，但要包含关键信息
  4. 添加元数据便于分类分析


## 2. 基于规则的评估器

### 核心价值

使用简单的Python函数检查输出的基本属性，快速、确定性、无需额外成本。

### 常见规则评估

| 评估项 | 说明 | 示例 |
|--------|------|------|
| **长度检查** | 输出长度是否合理 | 不能太短或太长 |
| **格式验证** | 是否符合预期格式 | JSON格式、包含特定字段 |
| **关键词检查** | 是否包含必需关键词 | 答案中必须提到"Python" |
| **禁用词检查** | 是否包含不应出现的词 | 不能包含脏话、敏感词 |

### 评估器函数签名

```python
def evaluator(inputs: dict, outputs: dict, reference: dict = None) -> dict:
    """
    评估器函数
    
    Args:
        inputs: 输入数据
        outputs: 模型输出
        reference: 参考答案（可选）
    
    Returns:
        {"score": float, "key": str, "comment": str}
    """
    pass
```

In [3]:
print("【演示2：基于规则的评估器】\n")

# 定义评估器1：长度检查
def length_evaluator(inputs: dict, outputs: dict, reference: dict = None) -> dict:
    """检查输出长度是否合理"""
    output_text = outputs.get("output", "")
    length = len(output_text)
    
    # 评分规则：20-200字符为合理范围
    if length < 20:
        score = 0.0
        comment = f"输出太短（{length}字符），可能信息不足"
    elif length > 200:
        score = 0.5
        comment = f"输出较长（{length}字符），可能冗余"
    else:
        score = 1.0
        comment = f"输出长度合理（{length}字符）"
    
    return {"key": "length", "score": score, "comment": comment}

# 定义评估器2：关键词检查
def keyword_evaluator(inputs: dict, outputs: dict, reference: dict = None) -> dict:
    """检查输出是否包含关键词"""
    output_text = outputs.get("output", "").lower()
    question = inputs.get("question", "").lower()
    
    # 根据问题确定必需关键词
    required_keywords = []
    if "python" in question:
        required_keywords = ["python", "语言"]
    elif "langchain" in question:
        required_keywords = ["langchain"]
    elif "rag" in question:
        required_keywords = ["rag", "检索"]
    
    # 检查关键词
    found_keywords = [kw for kw in required_keywords if kw in output_text]
    score = len(found_keywords) / len(required_keywords) if required_keywords else 1.0
    
    comment = f"找到 {len(found_keywords)}/{len(required_keywords)} 个关键词"
    
    return {"key": "keyword", "score": score, "comment": comment}

# 测试评估器
print("=== 测试评估器 ===\n")

# 模拟一个输出
test_output = {
    "output": "Python是一种高级编程语言，广泛用于数据科学和Web开发。"
}
test_input = {
    "question": "Python是什么类型的语言？"
}

# 运行评估器
length_result = length_evaluator(test_input, test_output)
keyword_result = keyword_evaluator(test_input, test_output)

print("长度评估：")
print(f"  分数: {length_result['score']}")
print(f"  评论: {length_result['comment']}\n")

print("关键词评估：")
print(f"  分数: {keyword_result['score']}")
print(f"  评论: {keyword_result['comment']}\n")

print("✓ 基于规则的评估器优点：")
print("  - 快速、确定性")
print("  - 无需额外API调用")
print("  - 适合检查基本属性")
print("✓ 局限性：")
print("  - 无法评估语义质量")
print("  - 规则需要人工定义")


【演示2：基于规则的评估器】

=== 测试评估器 ===

长度评估：
  分数: 1.0
  评论: 输出长度合理（31字符）

关键词评估：
  分数: 1.0
  评论: 找到 2/2 个关键词

✓ 基于规则的评估器优点：
  - 快速、确定性
  - 无需额外API调用
  - 适合检查基本属性
✓ 局限性：
  - 无法评估语义质量
  - 规则需要人工定义


## 3. LLM-as-Judge评估器

使用LLM作为评判者，评估输出的语义质量。

### 适用场景

- 相关性：答案是否回答了问题
- 准确性：答案是否与参考一致
- 有用性：答案是否有帮助
- 忠实度：答案是否基于检索内容


In [ ]:
print("【演示3：LLM-as-Judge评估器】\n")

# 定义评估提示词
RELEVANCE_PROMPT = """请评估以下答案是否回答了用户的问题。

问题: {question}
答案: {answer}

评分标准：
- 5分：完全回答了问题，信息准确完整
- 4分：基本回答了问题，信息大致正确
- 3分：部分回答了问题，有些信息缺失
- 2分：只有少量相关信息
- 1分：完全不相关

请只返回一个0-5的数字作为分数。"""

# 创建LLM-as-Judge评估器
def llm_relevance_evaluator(inputs: dict, outputs: dict, reference: dict = None) -> dict:
    """使用LLM评估答案的相关性"""
    question = inputs.get("question", "")
    answer = outputs.get("output", "")
    
    # 构造评估提示
    eval_prompt = RELEVANCE_PROMPT.format(question=question, answer=answer)
    
    # 调用LLM进行评估
    response = model.invoke(eval_prompt)
    
    # 解析分数（简单实现）
    try:
        score_text = response.content.strip()
        score = float(score_text[0]) / 5.0  # 归一化到0-1
        comment = f"LLM评分: {score_text[0]}/5"
    except:
        score = 0.5
        comment = "解析失败，默认0.5分"
    
    return {"key": "relevance", "score": score, "comment": comment}

# 测试LLM-as-Judge
print("=== 测试LLM-as-Judge评估器 ===\n")

test_cases = [
    {
        "input": {"question": "Python是什么类型的语言？"},
        "output": {"output": "Python是一种高级、解释型、面向对象的编程语言。"}
    },
    {
        "input": {"question": "Python是什么类型的语言？"},
        "output": {"output": "今天天气很好。"}  # 不相关的答案
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"测试案例 {i}:")
    print(f"  问题: {test_case['input']['question']}")
    print(f"  答案: {test_case['output']['output']}")
    
    result = llm_relevance_evaluator(test_case['input'], test_case['output'])
    
    print(f"  评估结果: {result['comment']}")
    print(f"  归一化分数: {result['score']:.2f}\n")

print("✓ LLM-as-Judge优点：")
print("  - 可以评估语义质量")
print("  - 灵活，可以评估各种指标")
print("  - 接近人类判断")
print("✓ 局限性：")
print("  - 需要额外的LLM调用（成本）")
print("  - 结果可能不稳定")
print("  - 需要精心设计评估提示词")

【演示3：LLM-as-Judge评估器】

=== 测试LLM-as-Judge评估器 ===

测试案例 1:
  问题: Python是什么类型的语言？
  答案: Python是一种高级、解释型、面向对象的编程语言。
  评估结果: LLM评分: 5/5
  归一化分数: 1.00

测试案例 2:
  问题: Python是什么类型的语言？
  答案: 今天天气很好。
  评估结果: LLM评分: 1/5
  归一化分数: 0.20

✓ LLM-as-Judge优点：
  - 可以评估语义质量
  - 灵活，可以评估各种指标
  - 接近人类判断
✓ 局限性：
  - 需要额外的LLM调用（成本）
  - 结果可能不稳定
  - 需要精心设计评估提示词


## 4. RAG评估指标

RAG应用需要评估检索和生成两个环节。

### RAG评估维度

| 维度 | 说明 | 需要参考答案 |
|------|------|-------------|
| **Document Relevance** | 检索的文档是否相关 | 否 |
| **Answer Faithfulness** | 答案是否基于检索内容 | 否 |
| **Answer Helpfulness** | 答案是否有帮助 | 否 |
| **Answer Correctness** | 答案是否正确 | 是 |

### 评估流程

```
问题 → RAG系统 → 检索文档 + 答案
         ↓           ↓         ↓
    评估检索质量  评估忠实度  评估准确性
```


In [7]:
print("【演示4：RAG评估示例】\n")

# 模拟RAG输出
rag_output = {
    "question": "什么是LangChain？",
    "retrieved_docs": [
        "LangChain是一个用于构建LLM应用的开源框架。",
        "LangChain提供了Agent、Chain等核心抽象。"
    ],
    "answer": "LangChain是一个开源框架，用于构建基于大语言模型的应用程序。"
}

# 评估器1：文档相关性
def doc_relevance_evaluator(rag_output: dict) -> dict:
    """评估检索文档是否相关"""
    question = rag_output["question"]
    docs = rag_output["retrieved_docs"]
    
    # 简单检查：文档是否包含问题中的关键词
    keywords = ["langchain"]
    relevant_docs = [doc for doc in docs if any(kw in doc.lower() for kw in keywords)]
    
    score = len(relevant_docs) / len(docs) if docs else 0.0
    comment = f"{len(relevant_docs)}/{len(docs)} 个文档相关"
    
    return {"key": "doc_relevance", "score": score, "comment": comment}

# 评估器2：答案忠实度
def answer_faithfulness_evaluator(rag_output: dict) -> dict:
    """评估答案是否基于检索内容"""
    answer = rag_output["answer"].lower()
    docs = rag_output["retrieved_docs"]
    
    # 检查答案中的关键信息是否来自文档
    # 简化实现：检查答案中的词是否在文档中
    answer_words = set(answer.split())
    doc_words = set(" ".join(docs).lower().split())
    
    overlap = len(answer_words & doc_words)
    score = overlap / len(answer_words) if answer_words else 0.0
    
    comment = f"答案中{overlap}/{len(answer_words)}个词来自检索文档"
    
    return {"key": "faithfulness", "score": score, "comment": comment}

# 运行评估
print("=== RAG评估示例 ===\n")
print(f"问题: {rag_output['question']}")
print(f"\n检索文档:")
for i, doc in enumerate(rag_output['retrieved_docs'], 1):
    print(f"  {i}. {doc}")
print(f"\n答案: {rag_output['answer']}\n")

# 评估
doc_rel = doc_relevance_evaluator(rag_output)
faithfulness = answer_faithfulness_evaluator(rag_output)

print("评估结果:")
print(f"  文档相关性: {doc_rel['score']:.2f} - {doc_rel['comment']}")
print(f"  答案忠实度: {faithfulness['score']:.2f} - {faithfulness['comment']}\n")

print("💡 RAG评估要点：")
print("  1. 检索质量直接影响最终答案")
print("  2. 答案必须基于检索内容（忠实度）")
print("  3. 需要评估多个维度")
print("  4. 生产环境建议使用LLM-as-Judge")

【演示4：RAG评估示例】

=== RAG评估示例 ===

问题: 什么是LangChain？

检索文档:
  1. LangChain是一个用于构建LLM应用的开源框架。
  2. LangChain提供了Agent、Chain等核心抽象。

答案: LangChain是一个开源框架，用于构建基于大语言模型的应用程序。

评估结果:
  文档相关性: 1.00 - 2/2 个文档相关
  答案忠实度: 0.00 - 答案中0/1个词来自检索文档

💡 RAG评估要点：
  1. 检索质量直接影响最终答案
  2. 答案必须基于检索内容（忠实度）
  3. 需要评估多个维度
  4. 生产环境建议使用LLM-as-Judge


## 5. Agent评估（轨迹评估）

Agent评估需要评估整个执行轨迹，包括工具调用顺序和决策。

### 轨迹评估方法

| 方法 | 说明 | 优点 | 缺点 |
|------|------|------|------|
| **Trajectory Match** | 与参考轨迹精确匹配 | 确定性、快速 | 不灵活 |
| **LLM-as-Judge** | LLM评估轨迹合理性 | 灵活、全面 | 成本高 |

### 评估维度

- 工具选择是否正确
- 工具调用顺序是否合理
- 参数是否正确
- 最终答案是否正确

In [8]:
print("【演示5：Agent轨迹评估】\n")

# 模拟Agent执行轨迹
agent_trajectory = {
    "question": "北京今天天气如何？",
    "steps": [
        {"type": "tool_call", "tool": "get_weather", "args": {"city": "北京"}},
        {"type": "tool_result", "result": "北京今天晴天，25°C"},
        {"type": "final_answer", "answer": "北京今天是晴天，气温25°C。"}
    ]
}

# 参考轨迹（期望的执行路径）
reference_trajectory = {
    "expected_tools": ["get_weather"],
    "expected_args": {"city": "北京"}
}

# 轨迹匹配评估器
def trajectory_match_evaluator(trajectory: dict, reference: dict) -> dict:
    """检查轨迹是否与参考匹配"""
    # 提取实际调用的工具
    actual_tools = [
        step["tool"] for step in trajectory["steps"] 
        if step["type"] == "tool_call"
    ]
    
    # 检查工具是否匹配
    expected_tools = reference["expected_tools"]
    tools_match = set(actual_tools) == set(expected_tools)
    
    # 检查参数
    tool_calls = [step for step in trajectory["steps"] if step["type"] == "tool_call"]
    args_match = all(
        step["args"] == reference["expected_args"] 
        for step in tool_calls
    )
    
    score = 1.0 if (tools_match and args_match) else 0.0
    comment = f"工具匹配: {tools_match}, 参数匹配: {args_match}"
    
    return {"key": "trajectory_match", "score": score, "comment": comment}

# 运行评估
print("=== Agent轨迹评估 ===\n")
print(f"问题: {agent_trajectory['question']}\n")
print("执行轨迹:")
for i, step in enumerate(agent_trajectory['steps'], 1):
    if step['type'] == 'tool_call':
        print(f"  {i}. 调用工具: {step['tool']}({step['args']})")
    elif step['type'] == 'tool_result':
        print(f"  {i}. 工具结果: {step['result']}")
    elif step['type'] == 'final_answer':
        print(f"  {i}. 最终答案: {step['answer']}")

result = trajectory_match_evaluator(agent_trajectory, reference_trajectory)

print(f"\n评估结果:")
print(f"  分数: {result['score']}")
print(f"  评论: {result['comment']}\n")

print("💡 Agent评估要点：")
print("  1. 不仅评估最终答案，还要评估过程")
print("  2. 轨迹匹配适合确定性场景")
print("  3. LLM-as-Judge适合灵活评估")
print("  4. 可以使用agentevals包进行专业评估")

【演示5：Agent轨迹评估】

=== Agent轨迹评估 ===

问题: 北京今天天气如何？

执行轨迹:
  1. 调用工具: get_weather({'city': '北京'})
  2. 工具结果: 北京今天晴天，25°C
  3. 最终答案: 北京今天是晴天，气温25°C。

评估结果:
  分数: 1.0
  评论: 工具匹配: True, 参数匹配: True

💡 Agent评估要点：
  1. 不仅评估最终答案，还要评估过程
  2. 轨迹匹配适合确定性场景
  3. LLM-as-Judge适合灵活评估
  4. 可以使用agentevals包进行专业评估


## 6. 评估最佳实践

### 评估策略选择

| 应用类型 | 推荐评估器 | 原因 |
|---------|-----------|------|
| **简单QA** | 基于规则 + LLM-as-Judge | 快速验证基本属性 + 语义质量 |
| **RAG应用** | 文档相关性 + 答案忠实度 + 准确性 | 多维度评估 |
| **Agent应用** | 轨迹评估 + 最终答案评估 | 评估过程和结果 |
| **对话系统** | 相关性 + 有用性 + 长度 | 用户体验导向 |

### 评估工作流

```python
# 1. 创建数据集
dataset = create_test_dataset()

# 2. 定义评估器
evaluators = [
    length_evaluator,
    keyword_evaluator,
    llm_relevance_evaluator
]

# 3. 运行评估
results = []
for example in dataset:
    # 运行应用
    output = app.invoke(example["input"])
    
    # 运行所有评估器
    scores = {}
    for evaluator in evaluators:
        result = evaluator(example["input"], output, example.get("reference"))
        scores[result["key"]] = result["score"]
    
    results.append({
        "input": example["input"],
        "output": output,
        "scores": scores
    })

# 4. 分析结果
avg_scores = {
    key: sum(r["scores"][key] for r in results) / len(results)
    for key in results[0]["scores"].keys()
}
```

### 关键原则

✅ **多维度评估** - 单一指标不足以衡量质量  
✅ **自动化优先** - 尽可能使用自动评估  
✅ **持续评估** - 在开发和生产中持续监控  
✅ **对比评估** - 用于比较不同版本  
✅ **人工抽查** - 定期人工验证自动评估结果


In [10]:
print("【演示6：完整评估流程】\n")

# 创建一个简单的QA应用
def simple_qa_app(question: str) -> str:
    """简单的QA应用"""
    response = model.invoke(question)
    return response.content

# 1. 准备测试数据集
test_dataset = [
    {"question": "Python是什么？", "category": "basic"},
    {"question": "如何学习编程？", "category": "advice"}
]

# 2. 定义多个评估器
evaluators = {
    "length": length_evaluator,
    "keyword": keyword_evaluator,
    "relevance": llm_relevance_evaluator
}

# 3. 运行评估
print("=== 运行评估 ===\n")
results = []

for i, example in enumerate(test_dataset, 1):
    print(f"评估样本 {i}/{len(test_dataset)}: {example['question'][:30]}...")
    
    # 运行应用
    output = simple_qa_app(example["question"])
    
    # 运行所有评估器
    scores = {}
    for eval_name, evaluator in evaluators.items():
        result = evaluator(
            {"question": example["question"]},
            {"output": output}
        )
        scores[eval_name] = result["score"]
        print(f"  {eval_name}: {result['score']:.2f}")
    
    results.append({
        "question": example["question"],
        "output": output[:50] + "...",
        "scores": scores
    })
    print()

# 4. 汇总结果
print("=== 评估汇总 ===\n")

# 计算平均分
avg_scores = {}
for eval_name in evaluators.keys():
    avg = sum(r["scores"][eval_name] for r in results) / len(results)
    avg_scores[eval_name] = avg
    print(f"{eval_name}: 平均分 {avg:.2f}")

print(f"\n总体平均分: {sum(avg_scores.values()) / len(avg_scores):.2f}\n")

print("💡 完整评估流程要点：")
print("  1. 使用多个评估器从不同角度评估")
print("  2. 在整个数据集上运行，而不是单个样本")
print("  3. 汇总统计结果，便于对比")
print("  4. 定期运行，监控性能变化")
print("  5. 结合自动评估和人工抽查")

【演示6：完整评估流程】

=== 运行评估 ===

评估样本 1/2: Python是什么？...
  length: 0.50
  keyword: 1.00
  relevance: 1.00

评估样本 2/2: 如何学习编程？...
  length: 0.50
  keyword: 1.00
  relevance: 1.00

=== 评估汇总 ===

length: 平均分 0.50
keyword: 平均分 1.00
relevance: 平均分 1.00

总体平均分: 0.83

💡 完整评估流程要点：
  1. 使用多个评估器从不同角度评估
  2. 在整个数据集上运行，而不是单个样本
  3. 汇总统计结果，便于对比
  4. 定期运行，监控性能变化
  5. 结合自动评估和人工抽查


## 7. 总结与关键收获

### 核心要点

#### 1. 评估的重要性

- LLM输出是非确定性的，需要系统化评估
- 评估确保质量、指导优化、建立信心
- 评估是迭代改进的基础

#### 2. 评估器类型

| 类型 | 适用场景 | 优点 | 缺点 |
|------|---------|------|------|
| **基于规则** | 基本属性检查 | 快速、确定 | 无法评估语义 |
| **LLM-as-Judge** | 语义质量评估 | 灵活、全面 | 成本高、不稳定 |
| **基于参考** | 准确性评估 | 客观 | 需要标注数据 |

#### 3. 不同应用的评估策略

```python
# QA应用
evaluators = [length, keyword, relevance, accuracy]

# RAG应用
evaluators = [doc_relevance, faithfulness, helpfulness, correctness]

# Agent应用
evaluators = [trajectory_match, tool_correctness, answer_quality]
```

#### 4. 评估 vs 测试

- **评估**：衡量性能，相对标准，用于对比
- **测试**：断言正确性，绝对标准，用于部署门槛
- 评估指标可以转换为测试（回归测试）

---

### 最佳实践

#### ✅ 推荐做法

1. **多维度评估**
   - 不依赖单一指标
   - 从不同角度评估质量

2. **自动化评估**
   - 优先使用自动评估器
   - 减少人工成本

3. **持续评估**
   - 开发阶段：每次修改后评估
   - 生产阶段：定期监控

4. **分层评估**
   - 快速评估：基于规则
   - 深度评估：LLM-as-Judge
   - 人工抽查：验证自动评估

5. **数据集管理**
   - 维护高质量测试集
   - 定期更新和扩充
   - 包含边界情况

#### ❌ 避免的陷阱

1. **只评估最终输出**
   - ❌ 忽略中间过程
   - ✅ 评估整个流程（特别是Agent）

2. **过度依赖单一指标**
   - ❌ 只看准确率
   - ✅ 多维度综合评估

3. **评估数据集太小**
   - ❌ 只有几个样本
   - ✅ 至少几十个样本

4. **不更新评估标准**
   - ❌ 一成不变的评估器
   - ✅ 随应用演进更新

5. **忽略成本**
   - ❌ 所有样本都用LLM-as-Judge
   - ✅ 平衡成本和质量

---

### 评估工具生态

| 工具 | 说明 | 适用场景 |
|------|------|---------|
| **LangSmith** | 官方评估平台 | 在线评估、团队协作 |
| **pytest** | 测试框架 | 本地测试、CI/CD |
| **agentevals** | Agent专用评估 | Agent轨迹评估 |
| **自定义评估器** | 灵活定制 | 特定需求 |

---

## 下一步学习

完成评估基础后，建议学习：

1. **第22章：LangSmith Integration** - 使用LangSmith进行在线评估
2. **综合项目** - 将评估集成到实际项目中
3. **高级评估** - A/B测试、多模型对比

---

## 关键收获

✅ **评估是LLM应用开发的核心环节** - 确保质量和持续改进

✅ **多种评估器组合使用** - 基于规则 + LLM-as-Judge + 人工

✅ **不同应用需要不同评估策略** - QA、RAG、Agent各有侧重

✅ **评估要覆盖过程和结果** - 特别是Agent应用

✅ **自动化评估提高效率** - 但需要人工验证

✅ **持续评估和迭代** - 在开发和生产中都要监控

✅ **评估指标可以转换为测试** - 建立质量门槛

✅ **LangSmith提供强大的评估工具** - 下一章将详细学习
